# Découvrons Pandas

Pandas est une librairie qui permet de préparer, arranger, combiner, et analyser de grands volumes de données. Pandas repose sur NumPy ce qui permet de travailler facilement avec des tableaux (Series dans Pandas) ou des matrices (DataFrames). Ces derniers sont plus *optimisés* que le type natif. Pandas et NumPy sont donc habituellement utilisés conjointement.

Note : la déclaration suivante sert à importer les fichiers ressources utilisés plus bas.

In [ ]:
comptage_path = '../assets/comptage-voyageurs-trains-transilien.csv'
museum_path = "../assets/frequentation-des-musees-de-france.csv"

## Installation de Pandas
Pandas est une librairie scientifique Python. Sous un système Linux, vous pouvez l’installer avec `pip`. Sous Windows ou Mac Os, vous utiliserez [Anaconda](https://www.anaconda.com).
Les modules qui utilisent Pandas commencent habituellement par les lignes :

In [ ]:
import pandas as pd
import numpy as np

## Pandas, les DataFrames, Series et les indexes
Dans Pandas, un `DataFrame` est une matrice qui peut être comparée à un tableur. Les lignes ou colonnes sont des `Series`.
Un index est une liste d’entiers (`int`) ou labels (`str`) qui identifie de manière unique une ligne ou une colonne. Les Series sont toujours indexées.
Une série est un objet créé à l’aide d’un constructeur qui accepte en paramètre une collection. La série créée de cette manière voit ses valeurs indexés avec un entier.

In [ ]:
pd.Series(range(1, 5))

L’index peut être spécifié ainsi que le nom de la série.

In [ ]:
pd.Series(range(1, 5), index=["un", "deux", "trois", "quatre"], name="ma_serie")

## Accéder à un élément
On peut accéder à un élément d’une série à partir de son label ou de son indice.

In [ ]:
my_serie = pd.Series(range(1, 5), index=["un", "deux", "trois", "quatre"], name="ma_serie")
print(my_serie[0])
print(my_serie['trois'])

Mais les labels peuvent aussi être des attributs

In [ ]:
my_serie.trois

Il est également possible d’utiliser le slicing avec les Series.

In [ ]:
print(my_serie['un' : 'trois'])
print(my_serie[0:3])

Pandas permet également l'accès par les indices spécifiques.

In [ ]:
print(my_serie[[0,3]])
print(my_serie[['un', 'trois', 'deux']])

On peut appliquer des fonctions méthématiques directement sur les Series et certaines sont disponibles sous forme de *raccourcis*.

In [ ]:
my_serie.sum()

On peut appliquer des opérations sur deux séries comme additionner deux séries, mais pour cela, leur indexes doivent être alignés.

In [ ]:
other_serie = pd.Series(np.random.rand(4), index=['trois', 'quatre', 'cinq', 'six'], name='Autre serie')

my_serie + other_serie

Pandas réalise l'union avant l'opération. Les valeurs manquantes ont été remplacées par un `NaN` (Not a number).

Pandas propose des méthodes permétant par exemple de remplacer les valeur manquantes (ici par 0)

In [ ]:
my_serie.add(other_serie, fill_value=0)

Mais si les indexes sont insignifiants et seul les valeurs nous intéressent, alors on peut supprimer les indexes

In [ ]:
my_serie.reset_index(drop=True) + other_serie.reset_index(drop=True)

## Créer des DataFrames à partir de séries
Un DataFrame est un *tableau*. On peut créer un DataFrame à partir de l'association de séries qui ont des indexes partagés. La classe DataFrame permet ceci par son constructeur.

In [ ]:
pd.DataFrame({'First':my_serie.reset_index(drop=True), 'Other':other_serie.reset_index(drop=True)})

## Manipuler des données

Un DataSet est disponible dans la librairie Seaborn

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

titanic = sns.load_dataset('titanic')

In [ ]:
titanic.head()

In [ ]:
titanic.embark_town.unique()

In [ ]:
titanic.age.unique()

La méthode `describe` donne un apperçu rapide (sous forme d'une `Series`) : le nombre de données, la moyenne globale, la déviation standard, le minimum, les quartiles et le maximum de la Serie.

In [ ]:
titanic.describe()

Pour les valeurs autre que numériques, `describe` retourne le nombre de données, le nombre d'éléments uniques, la valeur la plus fréquente et son nombre d'occurences. Lorsqu'il y a des données hétérogènes, `describe`n'affiche que les informations liées aux données quantifiées, on peut ajouter l'argument `include="all"`pour tout afficher.

In [ ]:
titanic.describe(include="all")

Il manque des données âge, nus pouvons leur donner une valeur spécifique.

In [ ]:
titanic.age.head(10)

In [ ]:
titanic.fillna(value={"age": 0}).age.head(10)

Ou la valeur précédente

In [ ]:
titanic.fillna(method="pad").age.head(10)

Ou supprimer la ligne

In [ ]:
titanic.dropna().head(10)

## Les *tableaux croisés dynamiques*
Pour voir la répartition des survivants en fonction de leurs sexes et de leur type de billet, nous n'avons besoin que d'une seule ligne

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class')

Les données sont groupées en fonction des critères que nous spécifions, et les résultats aggrégés en moyenne. Nous pouvons spécifier d'autres fonctions. Par exemple, si nous voulons savoir quelle est le nombre total de survivants dans chaque cas, nous utiliserons la fonction `sum`.

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', aggfunc="sum")

Nous pouvons aussi créer un dataframe multi-indexé où l'âge est une dimension suplémentaire grâce à la fonction `cut`.

In [ ]:
titanic.dropna(inplace=True)
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class')

## L'import de données
Pandas est capable d'importer des données à partie de la plupart des formats courants. Ci-dessous nous chargeons un fichier csv issu de l'OpenData de la SNCF en csv et affichons les 5 premières lignes.

In [ ]:
df = pd.read_csv(comptage_path,
            sep=";")
df.head()

Chaque colonne est une `Serie` et peut être accédée comme un élément du DataFrame

In [ ]:
df['Nom gare'].head()

Utilisons la méthode `describe` pour un apperçu rapide (sous forme d'une `Series`)

In [ ]:
df['Montants'].describe()

In [ ]:
df['Nom gare'].describe()

Pandas permet de trier les données

In [ ]:
df.sort_values("Nom gare")

Pandas permet de grouper les données

In [ ]:
df.groupby(['Nom gare', 'Type jour']).aggregate([np.mean, np.sum]).head(6)

Ici, nous avons la création d'un *MultiIndex* qui permet d'organiser les données par catégorie/niveau et précisions 6 lignes d'affichage pour avoir tous les éléments.

Pandas permet de créer de nouveaux `DataFrame` à partir des données d'un DataFrame.

In [ ]:
pd.DataFrame(df.groupby(['Nom gare'])['Montants'].count()) \
    .sort_values('Montants', ascending=False).head()

Pandas permet également d'extraire des informations de ce nouveau DataFrame

In [ ]:
df.groupby(['Nom gare'])['Montants'].describe()

Bien entendu, on peut réaliser des opérations sur ces extractions

In [ ]:
pd.DataFrame(df.groupby(['Nom gare'])['Montants'].sum()) \
    .sort_values('Montants', ascending=False).head()

In [ ]:
pd.DataFrame(df.groupby(['Date de comptage', 'Nom gare'])['Montants'].sum()) \
    .sort_values('Montants', ascending=False).head()

## Graphiques avec Matplotlib

In [ ]:
import matplotlib
import seaborn

matplotlib.rcParams['figure.figsize'] = 5, 4
seaborn.set_style('whitegrid')

In [ ]:
df['Montants'].plot()

In [ ]:
pd.DataFrame(df.groupby('Nom gare')['Montants'].sum()) \
    .sort_values('Montants', ascending=False).plot()